In [ ]:
#!import AutomatedAnalysis.dib

In [ ]:
string path = @".\Examples\Traces\All_Conserve";
DataManager dm = DataManager.CreateAspNetData(path);
string baselineConfigName = "28c";
string comparandConfigName = "28c_con1";
// Set to null if plot and summary all benchmarks
string[] benchmarkToSummary = ["ConnectionClose", "Stage2Pgo"]; 
// Set to null if plot and summary all iteration
int[] iterationToSummary = null;

List<Metric<TraceGC>> metricFilterList = ML(Metrics.G.GCPauseTime, Metrics.G.HeapSizeBefore, Metrics.G.HeapSizeAfter, Metrics.G.HeapSizePeak);

Filter runFilter = null;
Filter configFilter = null;


## Display Benchmark Regressions

In [ ]:
Metric<BenchmarkData> avgHeapCountChanges = Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Average);
Metric<BenchmarkData> avgGCCount = Metrics.Promote(Metrics.I.TotalGCCount, Aggregation.Average);
public static Dictionary<Metric<BenchmarkData>, ExpectationDirection> HigherMemoryLowerThroughput = new()
{
    { Metrics.B.AverageMaxHeapSize, ExpectationDirection.Increase },
    { Metrics.B.AverageP50Latency, ExpectationDirection.Decrease },
    /*
    Additionally, a metric with no change is also available. The entire enum is:
    public enum ExpectationDirection
    {
        Unknown,
        Increase,
        Decrease,
        NoChange,
    }
    */
};

Dictionary< Metric<BenchmarkData>, double> thresholds = new()
{
    { Metrics.B.AverageMaxHeapSize, 5 },
    { Metrics.B.AverageP50Latency, 5 },
};

DisplayBenchmarkSummaryWithRegressions(dm, baselineConfigName, comparandConfigName, metrics: ML( Metrics.B.AverageMaxHeapSize, Metrics.B.AverageP50Latency ), benchmarkRegressionThreshold: 5, benchmarksToDisplay: 5, 
metricToleranceOverridePercentage: thresholds,
expectationDirections: HigherMemoryLowerThroughput);

## Display Iteration Anomalies

In [ ]:
// Update Average -> Safe Average.
Aggregation SafeAverage = new Aggregation((gc => {
    double safeAverage;
    
    long count = 0;
    double sum = 0;

    foreach (var item in gc)
    {
        if (item != 0 && !double.IsNaN(item))
        {
            sum += item;
            count++;
        }
    }

    safeAverage = count == 0 ? double.NaN : sum / count;
    return safeAverage;

}), "SafeAverage", "");

Metric<IterationData> avgTCPToConsider = Metrics.Promote(Metrics.G.MedianThroughputCostPercent, SafeAverage);

Dictionary<Metric<IterationData>, double> volatilityOverrides = new()
{
    { Metrics.I.Gen0MeanPauseMSec, 10 },
    { Metrics.I.Latency50thMS, 20 },
};

// todo: override the volatility threshold
DisplayIterationData(dm, baselineConfigName, comparandConfigName, metrics: ML( avgTCPToConsider, Metrics.I.Gen0MeanPauseMSec, Metrics.I.Latency50thMS ) //, benchmarkFilter: new Filter("Stage1Pgo"),
,volatilityThreshold: 2, metricVolOverridePercentage: volatilityOverrides, iterationsToDisplay: 5) 

## Display GC data

In [ ]:
public class AveragedMetricData
{
    public Metric<TraceGC> Metric;
    public string RunName;
    public string ConfigName;
    public string BenchmarkName;
    public int IterationIndex;
    public double AveragedMetric;
}

List<AveragedMetricData> FlattenAndAverageTopLevelData(TopLevelData topLevelData)
{
    List<AveragedMetricData> flattenedAndAveragedTopLevelData = new();

    Dictionary<string, RunData> rdDict = runFilter == null ?
        topLevelData.Runs : topLevelData.Runs.Where(run => runFilter.Include(run.Key)).ToDictionary();

    foreach (var runName in rdDict.Keys)
    {
        RunData rd = rdDict[runName];
        Dictionary<string, ConfigData> cdDict = configFilter == null ?
            rd.Configs : rd.Configs.Where(config => configFilter.Include(config.Key)).ToDictionary();

        foreach (var configName in cdDict.Keys)
        {
            ConfigData cd = rd.Configs[configName];
            Dictionary<string, BenchmarkData> bdDict = benchmarkToSummary == null ?
                cd.Benchmarks : cd.Benchmarks.Where(benchmark => benchmarkToSummary.Contains(benchmark.Key)).ToDictionary();

            foreach (var benchmarkName in bdDict.Keys)
            {
                BenchmarkData bd = cd.Benchmarks[benchmarkName];

                List<(IterationData, int)> iterationWithIndexList = iterationToSummary == null ?
                    bd.Iterations.WithIndex().ToList() : 
                    bd.Iterations.WithIndex().Where((_, idx) => iterationToSummary.Contains(idx)).ToList();
                foreach (var (iterationData, iterationIdx) in iterationWithIndexList)    
                {
                    foreach (var metric in metricFilterList)
                    {
                        AveragedMetricData data = new()
                        {
                            Metric = metric,
                            RunName = runName,
                            ConfigName = configName,
                            BenchmarkName = benchmarkName,
                            IterationIndex = iterationIdx,
                            AveragedMetric = iterationData.GCProcessData.GCs
                                .Select(gc => metric.DoExtract(gc, 0))
                                .Average().Value
                        };
                        flattenedAndAveragedTopLevelData.Add(data);
                    }
                }
            }
        }
    }
    return flattenedAndAveragedTopLevelData;
}

List<AveragedMetricData> flattenedAndAveragedTopLevelData = FlattenAndAverageTopLevelData(dm.Data);

foreach (var metricDataGroup in flattenedAndAveragedTopLevelData.GroupBy(d => d.Metric))
{
    Metric<TraceGC> metric = metricDataGroup.Key;
    foreach (var benchmarkDataGroup in metricDataGroup.GroupBy(d => d.BenchmarkName))
    {
        string benchmarkName = benchmarkDataGroup.Key;
        Filter benchmarkFilter = new(benchmarkName);
        IntFilter iterationFilter = new(iterationToSummary);
        $"## Details for {metric.Title} of {benchmarkName}".DisplayAs("text/markdown");
        ChartGCData(dm, metric, 
                    runFilter: runFilter,
                    configFilter: configFilter,
                    benchmarkFilter: benchmarkFilter,
                    iterationFilter: iterationFilter)
            .ForEach(ch => ch.Display());

        $"## Summary for {metric.Title} of {benchmarkName}".DisplayAs("text/markdown");
        StringBuilder SummaryTableBuilder = new();
        SummaryTableBuilder.AppendLine($"| Run | Avg. {metric.Title}({metric.Unit}) |");
        SummaryTableBuilder.AppendLine("| -------- | --------: |");

        foreach (var configDataGroup in benchmarkDataGroup.GroupBy(d => d.ConfigName))
        {
            string configName = configDataGroup.Key;
            foreach (var iterationData in configDataGroup.OrderBy(d => d.IterationIndex))
            {
                SummaryTableBuilder.AppendLine(
                    $"| {iterationData.ConfigName}_{iterationData.IterationIndex} | {iterationData.AveragedMetric.ToString("f2")} |");
            }
        }
        SummaryTableBuilder.ToString().DisplayAs("text/markdown");
    }
}
